Load Library

In [2]:
from keras.applications.xception import Xception
from keras.preprocessing import image
import numpy as np
from keras.layers import Dense, Dropout
from keras.models import Model
import os
from keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from keras.metrics import categorical_accuracy
from keras.callbacks import Callback, ModelCheckpoint,LearningRateScheduler,EarlyStopping,ReduceLROnPlateau
from keras.optimizers import SGD,Adam

Load Driver

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Load data set

In [3]:
def Load_data(fea):
  X_train=[]
  Y_train=[]
  X_test=[]
  Y_test=[]
  s_weight='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/'+str(fea)+'_weight.h5'
  for s in range(1,10):
    path='/content/drive/MyDrive/srtp-xsh/RGBimg_with_feature/s'+str(s)+'/'+str(fea)
    
    train=path+'/train'
    test=path+'/test'
    for x in os.listdir(train):
      f=train+'/'+x
      img=image.load_img(f)
      img=image.img_to_array(img)
      img=image.smart_resize(img, (100,200), interpolation='bilinear')
      label=x[3]
      X_train.append(img)
      Y_train.append(label)
    for x in os.listdir(test):
      f=test+'/'+x
      img=image.load_img(f)
      img=image.img_to_array(img)
      img=image.smart_resize(img, (100,200), interpolation='bilinear')
      label=x[3]
      X_test.append(img)
      Y_test.append(label)
  
  X_train=np.asarray(X_train,np.float32)
  X_test=np.asarray(X_test,np.float32)
  Y_train=np.asarray(Y_train,np.float32)
  Y_test=np.asarray(Y_test,np.float32)
  num_example=X_train.shape[0]
  arr=np.arange(num_example)
  np.random.shuffle(arr)
  X_train=X_train[arr]
  Y_train=Y_train[arr]
  Y_train = to_categorical(Y_train,num_classes=52)
  Y_test = to_categorical(Y_test,num_classes=52)
  print('shape of training data:',X_train.shape)
  print('shape of training label:',Y_train.shape)
  print('shape of test data:',X_test.shape)
  print('shape of test label:',Y_test.shape)
  return X_train,Y_train,X_test,Y_test,s_weight



In [5]:
X_train,Y_train,X_test,Y_test,s_weight=Load_data('IEMG')

shape of training data: (3276, 100, 200, 3)
shape of training label: (3276, 52)
shape of test data: (1404, 100, 200, 3)
shape of test label: (1404, 52)


Load model

In [6]:
base_model=Xception(include_top=False, weights='imagenet', input_tensor=None, input_shape=(100,200,3), pooling='avg')
x=base_model.output
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(52, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

83697664/83683744 [==============================] - 1s 0us/step


overview of performance

In [7]:
opt=Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['categorical_accuracy']) 
callbacks = [ModelCheckpoint(s_weight, monitor='val_loss', save_best_only=True),learning_rate_reduction]
history = model.fit(         X_train,
                    Y_train,
                    batch_size=32,
                    epochs=24,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/24
93/93 [==============================] - 83s 300ms/step - loss: 1.9772 - categorical_accuracy: 0.3583 - val_loss: 2.3887 - val_categorical_accuracy: 0.4024


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/24
93/93 [==============================] - 23s 252ms/step - loss: 1.1195 - categorical_accuracy: 0.6229 - val_loss: 1.1267 - val_categorical_accuracy: 0.6037
Epoch 3/24
93/93 [==============================] - 23s 248ms/step - loss: 0.9000 - categorical_accuracy: 0.6983 - val_loss: 1.8839 - val_categorical_accuracy: 0.5610
Epoch 4/24
93/93 [==============================] - 23s 251ms/step - loss: 0.6934 - categorical_accuracy: 0.7731 - val_loss: 0.9881 - val_categorical_accuracy: 0.6585
Epoch 5/24
93/93 [==============================] - 23s 251ms/step - loss: 0.6784 - categorical_accuracy: 0.7914 - val_loss: 1.8138 - val_categorical_accuracy: 0.5610
Epoch 6/24
93/93 [==============================] - 23s 248ms/step - loss: 0.4812 - categorical_accuracy: 0.8317 - val_loss: 1.4463 - val_categorical_accuracy: 0.6768
Epoch 7/24
93/93 [==============================] - 23s 249ms/step - loss: 0.3859 - categorical_accuracy: 0.8720 - val_loss: 1.0914 - val_categorical_accuracy: 0.756

In [1]:
model.load_weights(s_weight)
loss, accuracy = model.evaluate(X_test,
                  Y_test)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))

NameError: ignored